In [1]:
!git clone "https://github.com/jik876/hifi-gan.git"

Cloning into 'hifi-gan'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48 (from 1)
Receiving objects: 100% (48/48), 620.94 KiB | 13.21 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
cd hifi-gan

/kaggle/working/hifi-gan


In [3]:
# import os
# import torch
# import torchaudio
# import numpy as np
# from tqdm import tqdm

# # Config values (taken from your config.json)
# sampling_rate = 22050
# n_fft = 1024
# win_size = 1024
# hop_size = 256
# num_mels = 80
# fmin = 0
# fmax = 8000

# # Input/output dirs
# input_dir = "/kaggle/input/dir4-singfox/Directory 4/songs/Hindi"         # <-- Update this
# output_dir = "/kaggle/working/hifi-gan/test_mel_files/Hindi"     # <-- Update this
# os.makedirs(output_dir, exist_ok=True)

# # Mel Spectrogram extractor
# mel_transform = torchaudio.transforms.MelSpectrogram(
#     sample_rate=sampling_rate,
#     n_fft=n_fft,
#     win_length=win_size,
#     hop_length=hop_size,
#     f_min=fmin,
#     f_max=fmax,
#     n_mels=num_mels,
#     power=1.0,
#     normalized=False
# )

# # Function: 16kHz WAV → log-mel → .npy
# def wav_to_mel(wav_path):
#     waveform, orig_sr = torchaudio.load(wav_path)
    
#     # Mono
#     if waveform.size(0) > 1:
#         waveform = waveform.mean(dim=0, keepdim=True)

#     # Resample to 22.05 kHz
#     if orig_sr != sampling_rate:
#         waveform = torchaudio.functional.resample(waveform, orig_sr, sampling_rate)

#     # Mel spectrogram → log-mel
#     mel = mel_transform(waveform)
#     mel = torch.log(torch.clamp(mel, min=1e-5))  # avoid log(0)
#     return mel.squeeze(0).cpu().numpy()

# # Process all files
# for fname in tqdm(os.listdir(input_dir)):
#     if not fname.endswith('.wav'):
#         continue

#     wav_path = os.path.join(input_dir, fname)
#     mel = wav_to_mel(wav_path)
#     npy_path = os.path.join(output_dir, fname.replace('.wav', '.npy'))
#     np.save(npy_path, mel)
#     print(f"Saved: {npy_path}")


In [4]:
# import glob
# import os
# import numpy as np
# import json
# import torch
# from scipy.io.wavfile import write
# from env import AttrDict
# from meldataset import MAX_WAV_VALUE
# from models import Generator

# # --- Modify paths here ---
# input_mels_dir = "/kaggle/working/hifi-gan/test_mel_files/Hindi"  # <- folder of .npy mel files
# output_dir = "/kaggle/working/Hindi_gen"
# checkpoint_file = "/kaggle/input/other-hifi-models/generator_v3"
# config_file = "/kaggle/input/other-hifi-models/config (1).json"
# # --------------------------

# # Load config
# with open(config_file) as f:
#     json_config = json.load(f)
# h = AttrDict(json_config)

# # Set device
# torch.manual_seed(h.seed)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if device.type == "cuda":
#     torch.cuda.manual_seed(h.seed)

# # Load Generator
# generator = Generator(h).to(device)
# checkpoint_dict = torch.load(checkpoint_file, map_location=device)
# generator.load_state_dict(checkpoint_dict['generator'])
# generator.eval()
# generator.remove_weight_norm()

# # Inference
# os.makedirs(output_dir, exist_ok=True)
# filelist = sorted(os.listdir(input_mels_dir))

# with torch.no_grad():
#     for filname in filelist:
#         if not filname.endswith(".npy"):
#             continue
#         mel_path = os.path.join(input_mels_dir, filname)
#         x = np.load(mel_path)
#         x = torch.FloatTensor(x).unsqueeze(0).to(device)  # shape: (1, 80, T)
#         y_g_hat = generator(x)
#         audio = y_g_hat.squeeze().cpu().numpy()
#         audio = audio * MAX_WAV_VALUE
#         audio = audio.astype('int16')
#         output_path = os.path.join(output_dir, os.path.splitext(filname)[0] + ".wav")
#         write(output_path, h.sampling_rate, audio)
#         print("Saved:", output_path)


In [5]:
# !zip -r Hindi_file.zip /kaggle/working/Hindi_gen
# from IPython.display import FileLink
# FileLink(r'Hindi_file.zip')

In [6]:
import os
import torch
import torchaudio
import numpy as np
import json
from tqdm import tqdm
from scipy.io.wavfile import write
from env import AttrDict
from meldataset import MAX_WAV_VALUE
from models import Generator
from IPython.display import FileLink

# === CONFIG ===
sampling_rate = 22050
n_fft = 1024
win_size = 1024
hop_size = 256
num_mels = 80
fmin = 0
fmax = 8000

# === PATHS ===
base_input_dir = "/kaggle/input/singfox-dataset/dir5_singfox/Directory 5/songs"
base_mel_dir = "/kaggle/working/hifi-gan/test_mel_files"
base_output_dir = "/kaggle/working/generated_audio"
checkpoint_file = "/kaggle/input/other-hifi-models/generator_v3"
config_file = "/kaggle/input/other-hifi-models/config (1).json"

# === LOAD MODEL CONFIG ===
with open(config_file) as f:
    json_config = json.load(f)
h = AttrDict(json_config)

# === DEVICE ===
torch.manual_seed(h.seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    torch.cuda.manual_seed(h.seed)

# === LOAD GENERATOR ===
generator = Generator(h).to(device)
checkpoint_dict = torch.load(checkpoint_file, map_location=device)
generator.load_state_dict(checkpoint_dict['generator'])
generator.eval()
generator.remove_weight_norm()

# === MEL TRANSFORM ===
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sampling_rate,
    n_fft=n_fft,
    win_length=win_size,
    hop_length=hop_size,
    f_min=fmin,
    f_max=fmax,
    n_mels=num_mels,
    power=1.0,
    normalized=False
)

# === FUNCTION TO CONVERT WAV TO MEL ===
def wav_to_mel(wav_path):
    waveform, orig_sr = torchaudio.load(wav_path)
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if orig_sr != sampling_rate:
        waveform = torchaudio.functional.resample(waveform, orig_sr, sampling_rate)
    mel = mel_transform(waveform)
    mel = torch.log(torch.clamp(mel, min=1e-5))
    return mel.squeeze(0).cpu().numpy()

# === PROCESS LANGUAGES ===
languages = sorted([lang for lang in os.listdir(base_input_dir) if os.path.isdir(os.path.join(base_input_dir, lang))])
for language in tqdm(languages, desc="Processing Languages"):
    input_lang_dir = os.path.join(base_input_dir, language)
    mel_lang_dir = os.path.join(base_mel_dir, language)
    output_lang_dir = os.path.join(base_output_dir, language)

    os.makedirs(mel_lang_dir, exist_ok=True)
    os.makedirs(output_lang_dir, exist_ok=True)

    # === MEL EXTRACTION ===
    wav_files = [f for f in os.listdir(input_lang_dir) if f.endswith('.wav')]
    for fname in tqdm(wav_files, desc=f"Mel → {language}", leave=False):
        wav_path = os.path.join(input_lang_dir, fname)
        mel = wav_to_mel(wav_path)
        np.save(os.path.join(mel_lang_dir, fname.replace('.wav', '.npy')), mel)

    # === INFERENCE ===
    mel_files = sorted([f for f in os.listdir(mel_lang_dir) if f.endswith('.npy')])
    with torch.no_grad():
        for fname in tqdm(mel_files, desc=f"HiFi-GAN → {language}", leave=False):
            mel_path = os.path.join(mel_lang_dir, fname)
            x = np.load(mel_path)
            x = torch.FloatTensor(x).unsqueeze(0).to(device)
            y_g_hat = generator(x)
            audio = y_g_hat.squeeze().cpu().numpy()
            audio = (audio * MAX_WAV_VALUE).astype('int16')
            output_path = os.path.join(output_lang_dir, fname.replace('.npy', '.wav'))
            write(output_path, h.sampling_rate, audio)

    # === ZIP OUTPUT ===
    zip_path = f"/kaggle/working/{language}_generated.zip"
    os.system(f"zip -r -q {zip_path} {output_lang_dir}")  # -q for quiet zip
    print(f"\n✅ Zipped: {zip_path}")
    display(FileLink(zip_path))


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Removing weight norm...


Mel → Bengali:  99%|█████████▉| 497/500 [00:09<00:00, 61.22it/s]
                                                                
HiFi-GAN → Bengali:  97%|█████████▋| 486/500 [00:03<00:00, 169.67it/s]
                                                                      


✅ Zipped: /kaggle/working/Bengali_generated.zip


/kaggle/working/Bengali_generated.zip

...

/kaggle/working/Turkish_generated.zip

Mel → Vietnamese: 100%|█████████▉| 499/500 [00:07<00:00, 63.42it/s]
                                                                   
HiFi-GAN → Vietnamese:  97%|█████████▋| 486/500 [00:02<00:00, 168.01it/s]
                                                                         


✅ Zipped: /kaggle/working/Vietnamese_generated.zip


/kaggle/working/Vietnamese_generated.zip

Processing Languages: 100%|██████████| 20/20 [04:04<00:00, 12.25s/it]


In [7]:
print("Hi!!!")

Hi!!!
